# Question 1

### Exercise 1.(b)

In [418]:
#1-hidden layer neural network y=w2^T*tanh(w1*x+b1)+b2
#W1: 20 *10
#W2: 1 * 20

import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time
import torch

#generate data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

h_1 = 20
h_2 = 1
param_dict = {"W1": torch.randn(h_1,10,device=device,requires_grad=True),
              "b1":torch.randn(h_1,1,device=device,requires_grad=True),
              "W2":torch.randn(h_1,h_2,device=device,requires_grad=True),
              "b2":torch.randn(1, device=device,requires_grad=True)}

#generate data
x = torch.randn(100,10,1,device=device)
y = torch.randn(100,1,1,device=device)


def my_nn(x,param_dict):
    
    x = x.clone().detach().requires_grad_(True)
    x = torch.tanh(param_dict["W1"]@x+param_dict["b1"])
    x = param_dict["W2"].T@x+param_dict["b2"]
    return x

def my_MAE(y_hat,y):
    # abs value
    return torch.mean(torch.abs(y_hat-y))

def my_loss_grad(y_hat,y):
    return torch.sign(y_hat-y)

def my_nn_grad(x,y_hat,y,param_dict):

    x = x.clone().detach().requires_grad_(True)
    y_hat = y_hat.clone().detach().requires_grad_(True)
    y = y.clone().detach().requires_grad_(True)
    
    grad_dict = {}
    mlg = my_loss_grad(y_hat,y)
    grad_dict["b2"] = torch.mean(mlg)
    # my_loss_grad(y_hat,y) is 100*1*1
    # torch.tanh(param_dict["W1"]@x+param_dict["b1"]) is 100*20*1
    pre_w2 = mlg * torch.tanh(param_dict["W1"]@x+param_dict["b1"])
    grad_dict["W2"] = torch.mean(pre_w2,dim=0)
    pre_b1 = mlg * param_dict["W2"] * (1-torch.tanh(param_dict["W1"]@x+param_dict["b1"])**2)
    grad_dict["b1"] = torch.mean(pre_b1,dim=0)
    pre_w1 = pre_b1 @ x.permute(0,2,1)
    grad_dict["W1"] = torch.mean(pre_w1,dim=0)

    return grad_dict

def torch_grad_check(x,y,param_dict,eps=1e-6):
    # check my_nn_grad and torch.autograd.grad
    print("Floating point error tolerance is: ", eps)
    y_hat = my_nn(x,param_dict)
    #backwards
    loss = my_MAE(y_hat,y)
    loss.backward()
    grad_dict = my_nn_grad(x,y_hat,y,param_dict)
    for key in param_dict.keys():
        #use torch.linalg.norm to compare if smaller than eps
        print ("Key is: ", key)
        #print ("My grad is: ", grad_dict[key])
        #print ("Torch grad is: ", param_dict[key].grad)
        diff = torch.linalg.norm(grad_dict[key]-param_dict[key].grad)
        print ("Difference is: ", diff)
        #assert diff < eps, print error message if failed, print success message if passed
        assert diff < eps, "error in "+key
        print("Success in "+key)

   



#check my_nn_grad and torch.autograd
torch_grad_check(x,y,param_dict)






Floating point error tolerance is:  1e-06
Key is:  W1
Difference is:  tensor(1.1212e-07, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
Success in W1
Key is:  b1
Difference is:  tensor(5.2760e-08, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
Success in b1
Key is:  W2
Difference is:  tensor(9.1290e-08, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
Success in W2
Key is:  b2
Difference is:  tensor(0., device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)
Success in b2


### Exercise 1.(c)

In [419]:
# Train this model on the sklearn California Housing Prices datasets
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html#sklearn.datasets.fetch_california_housing

import scipy
import sklearn
import sklearn.datasets
import sklearn.linear_model
from torch.utils import data


# Generate a random California housing dataset
# half of the data is used for training, the other half for testing
# the data is normalized to have zero mean and unit variance
scaler = sklearn.preprocessing.StandardScaler()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    scaler.fit_transform(sklearn.datasets.fetch_california_housing().data),
    sklearn.datasets.fetch_california_housing().target,
    test_size=0.5,
    random_state=42)

train_batch_size = 64
test_batch_size = 64

# dataloader to gpu
train_dataset = data.TensorDataset(torch.tensor(X_train,device=device, dtype=torch.float32), torch.tensor(y_train,device=device, dtype=torch.float32))
train_loader = data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_dataset = data.TensorDataset(torch.tensor(X_test,device=device, dtype=torch.float32), torch.tensor(y_test,device=device, dtype=torch.float32))
test_loader = data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

#validation set from training set
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [int(0.8*len(train_dataset)), int(0.2*len(train_dataset))])
val_loader = data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=True)



# convenient xavier initialization, taking an existing dictionary of parameters

def xavier_init(param_dict):
    for key in param_dict.keys():
        if "W" in key:
            torch.nn.init.xavier_uniform_(param_dict[key])
        else:
            torch.nn.init.zeros_(param_dict[key])
        param_dict[key].requires_grad_(True)
    return param_dict

num_epochs = 40
optimizer = torch.optim.Adam(param_dict.values(), lr=0.001)
lr_scehudler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
loss_fn = torch.nn.MSELoss()


#change w1 to 20*8 instead of 20*10
param_dict["W1"] = torch.randn(h_1,8,device=device,requires_grad=True)
param_dict = xavier_init(param_dict)


train_loss_per_epoch = []
val_loss_per_epoch = []
test_loss_per_epoch = []

for epoch in range(num_epochs):
    train_loss = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        y_hat = my_nn(x.T,param_dict)
        y_hat = y_hat.squeeze()
        loss = loss_fn(y_hat,y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss_per_epoch.append(train_loss/len(train_loader))

    #validation
    val_loss = 0
    for batch_idx, (x, y) in enumerate(val_loader):
        y_hat = my_nn(x.T,param_dict)
        y_hat = y_hat.squeeze()
        loss = loss_fn(y_hat,y)
        val_loss += loss.item()
    val_loss_per_epoch.append(val_loss/len(val_loader))
    
    #test
    test_loss = 0
    for batch_idx, (x, y) in enumerate(test_loader):
        y_hat = my_nn(x.T,param_dict)
        y_hat = y_hat.squeeze()
        loss = loss_fn(y_hat,y)
        test_loss += loss.item()
    test_loss_per_epoch.append(test_loss/len(test_loader))

    lr_scehudler.step(val_loss_per_epoch[-1])

    print("Epoch: ", epoch+1, "Train loss: ", train_loss_per_epoch[-1], "Val loss: ", val_loss_per_epoch[-1], "Test loss: ", test_loss_per_epoch[-1])



Epoch:  1 Train loss:  4.929087238547242 Val loss:  3.2611297332879268 Test loss:  3.2259187286282764
Epoch:  2 Train loss:  2.0934204860233967 Val loss:  1.3236310229156956 Test loss:  1.2724719422834891
Epoch:  3 Train loss:  0.9945494738625892 Val loss:  0.8580863909287886 Test loss:  0.8100081691403448
Epoch:  4 Train loss:  0.7692363986998428 Val loss:  0.7691951457298163 Test loss:  0.7115339984496435
Epoch:  5 Train loss:  0.7048417037283933 Val loss:  0.7310889318133845 Test loss:  0.6757433416298878
Epoch:  6 Train loss:  0.6804037222891678 Val loss:  0.7329685986042023 Test loss:  0.6562707655959659
Epoch:  7 Train loss:  0.6602157961439203 Val loss:  0.7022379772229628 Test loss:  0.6451992902122898
Epoch:  8 Train loss:  0.6530472982076951 Val loss:  0.6851504544417063 Test loss:  0.6329366741357026
Epoch:  9 Train loss:  0.6383828653229607 Val loss:  0.6795233605485974 Test loss:  0.6215177156307079
Epoch:  10 Train loss:  0.6264210519967256 Val loss:  0.6677028925129862 T

In [428]:
def generate_dict(L,D,K,P):
    param_dict = {}
    param_dict["W1"] = torch.randn(K,D,device=device)
    for i in range(2,L+1):
        param_dict["W"+str(i)] = torch.randn(K,K,device=device)
    param_dict["WF"] = torch.randn(P,K,device=device)
    return param_dict

def my_nn_2a(x,L,param_dict):
    x = param_dict["W1"]@x
    for i in range(2,L+1):
        x = param_dict["W"+str(i)]@torch.tanh(x)
    x = param_dict["WF"]@torch.tanh(x)
    return x

# backward automatic differentiation from scratch
# using the chain rule

def my_backward_2a(x,L,param_dict):
    P = param_dict["WF"].shape[0]
    D = x.shape[0]
    #forward pass
    x = x.clone().detach().requires_grad_(True)
    x = param_dict["W1"]@x
    tanh_outputs =[x]
    for i in range(2,L+1):
        x = param_dict["W"+str(i)]@torch.tanh(x)
        tanh_outputs.append(x)
    x = param_dict["WF"]@torch.tanh(x)
    #backward pass for jacobian
    df_dx = torch.zeros((P,D),device=device)
    df_intermediate = torch.diag(1-torch.tanh(tanh_outputs[-1])**2)
    df_intermediate = param_dict["WF"]@df_intermediate
    for i in range(L,1,-1):
        df_intermediate = df_intermediate@ param_dict["W"+str(i)]@torch.diag(1-torch.tanh(tanh_outputs[i-2])**2)
    df_intermediate = df_intermediate@param_dict["W1"]
    #copy df_intermediate to df_dx
    df_dx = df_intermediate.clone().detach()
    return df_dx


 
    

epsilon = 1e-2
    

    
D = 2
K= 30
P = 10
L = 10
param_dict = generate_dict(L,D,K,P)

start = time.time()
for i in range(1000):
    test = torch.randn(D,device=device,requires_grad=True)
    my_J = my_backward_2a(test,L,param_dict)
    J_autograd = torch.autograd.functional.jacobian(lambda x: my_nn_2a(x,L,param_dict),test,create_graph=True,strategy="reverse-mode")
    #assert, print two jacobians if they are not equal
    assert torch.allclose(my_J,J_autograd,atol=epsilon), print(my_J,J_autograd)
    #Get time taken
end = time.time()
print("Time taken for 1000 iterations: ", end-start) 
print ("Jacobian is correct")
    

    
        

        
    
        


        

Time taken for 1000 iterations:  9.357636213302612
Jacobian is correct


### Exercise 2.(b)

In [429]:
def my_forward_2b(x,L,param_dict):
    x = param_dict["W1"]@x
    df_dx = param_dict["W1"]
    for i in range(2,L+1):
        x = torch.tanh(x)
        #foward automatic differentiation
        df_intermediate = torch.diag(1-x**2)
        df_dx = param_dict["W"+str(i)]@df_intermediate@df_dx
        x = param_dict["W"+str(i)]@x
    x = torch.tanh(x)
    df_intermediate = torch.diag(1-x**2)
    df_dx = param_dict["WF"]@df_intermediate@df_dx
    x = param_dict["WF"]@x
    return df_dx

param_dict = generate_dict(L,D,K,P)

start = time.time()
for i in range(1000):
    test_2b = torch.randn(D,device=device,requires_grad=True)
    my_J_2b = my_forward_2b(test_2b,L,param_dict)
    J_autograd_2b = torch.autograd.functional.jacobian(lambda x: my_nn_2a(x,L,param_dict),test_2b, strategy="forward-mode", vectorize=True)
    #assert, print two jacobians if they are not equal
    assert torch.allclose(my_J_2b,J_autograd_2b,atol=epsilon), print(my_J_2b,J_autograd_2b)
end = time.time()
print("Time taken for 1000 iterations: ", end-start)
print ("Jacobian is correct")


Time taken for 1000 iterations:  4.432066440582275
Jacobian is correct


### Exercise 2(c)

In [436]:
#Run on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on GPU: ", torch.cuda.is_available())

test_l = [3,5,10]

for L in test_l:
    print("L = ", L)
    param_dict = generate_dict(L,D,K,P)
    start = time.time()
    for i in range(10000):
        test = torch.randn(D,device=device,requires_grad=True)
        my_J = my_backward_2a(test,L,param_dict)
    print ("Time taken for 10000 iterations of backward with L = ", L, " is ", time.time()-start)
    start = time.time()
    for i in range(10000):
        test_2b = torch.randn(D,device=device,requires_grad=True)
        my_J_2b = my_forward_2b(test_2b,L,param_dict)
    print ("Time taken for 10000 iterations of forward with L = ", L, " is ", time.time()-start)
 


Running on GPU:  True
L =  3
Time taken for 10000 iterations of backward with L =  3  is  5.555159330368042
Time taken for 10000 iterations of forward with L =  3  is  4.441255569458008
L =  5
Time taken for 10000 iterations of backward with L =  5  is  8.366724491119385
Time taken for 10000 iterations of forward with L =  5  is  6.997880697250366
L =  10
Time taken for 10000 iterations of backward with L =  10  is  15.488043308258057
Time taken for 10000 iterations of forward with L =  10  is  13.669673681259155


In [437]:
#Run on GPU
device = "cpu"
print("Running on CPU")

test_l = [3,5,10]

for L in test_l:
    print("L = ", L)
    param_dict = generate_dict(L,D,K,P)
    start = time.time()
    for i in range(10000):
        test = torch.randn(D,device=device,requires_grad=True)
        my_J = my_backward_2a(test,L,param_dict)
    print ("Time taken for 10000 iterations of backward with L = ", L, " is ", time.time()-start)
    start = time.time()
    for i in range(10000):
        test_2b = torch.randn(D,device=device,requires_grad=True)
        my_J_2b = my_forward_2b(test_2b,L,param_dict)
    print ("Time taken for 10000 iterations of forward with L = ", L, " is ", time.time()-start)
 

Running on CPU
L =  3
Time taken for 10000 iterations of backward with L =  3  is  2.355909585952759
Time taken for 10000 iterations of forward with L =  3  is  2.016078472137451
L =  5
Time taken for 10000 iterations of backward with L =  5  is  3.727365493774414
Time taken for 10000 iterations of forward with L =  5  is  3.2781364917755127
L =  10
Time taken for 10000 iterations of backward with L =  10  is  6.791688680648804
Time taken for 10000 iterations of forward with L =  10  is  6.134467840194702
